In [27]:
!git clone https://github.com/Chris10M/OpML.git

fatal: destination path 'OpML' already exists and is not an empty directory.


In [28]:
import sys
sys.path.append('./OpML')

In [35]:
import numpy as np
from tqdm.notebook import tqdm
import os
import csv
import torch
from torchvision import datasets, transforms
from torchvision.utils import save_image
from args import args, kwargs

from optimizer import Optimizers
from models.vae_discrete import VAE as VAE_discrete
from models.vae_gumel_softmax import VAE_gumbel
from models.vae_reinforce import VAE as VAE_reinforce


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
MODELS = {
    'discrete': VAE_discrete,
    'gumbel': VAE_gumbel,
    'reinforce': VAE_reinforce,
}

In [37]:
def train(model, optimizer, train_loader, epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        data = data.to(device) 
        optimizer.zero_grad()
        outs = model(data)
        loss = model.loss_function(*outs, data)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    return train_loss / len(train_loader.dataset)


def test(model, test_loader):
    model.eval()

    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(tqdm(test_loader)):
            data = data.to(device)
            outs = model(data)
            test_loss += model.loss_function(*outs, data).item()

    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [38]:
def run(model, optimizer, train_loader, test_loader): 
    metrics = dict()  
    for epoch in range(1, args.epochs + 1):    
        train(model, optimizer, train_loader, epoch)
        test_loss = test(model, test_loader)
        samples = model.sample(device)

        metrics[epoch] = {
            'loss': test_loss,
            'samples': samples,
        }

    return metrics

In [39]:
def main():
    train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

    for model_name, model in MODELS.items():
        os.makedirs(f'results/{model_name}', exist_ok=True)

        for optim_name, optimizer in Optimizers.items():
            os.makedirs(f'results/{model_name}/{optim_name}/image', exist_ok=True)

            model_instance = model()
            metrics = run(model_instance.to(device), optimizer(params=model_instance.parameters()), train_loader, test_loader)

            losses = list()
            for epoch, item in metrics.items():
                losses.append([epoch, item['loss']])
                save_image(item['samples'], f'results/{model_name}/{optim_name}/image/{epoch}.png', nrow=8)

            with open(f'results/{model_name}/{optim_name}/test_loss.csv', 'w', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerows(losses)

In [ ]:
main()

  0%|          | 0/469 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/469 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
